In [186]:
import pandas as pd
import sklearn

In [216]:
features = pd.read_pickle("../data/processed/features.pkl")
features.head()

,installation_id,12 Monkeys,Air Show,All Star Sorting,Balancing Act,Bird Measurer (Assessment),Bottle Filler (Activity),Bubble Bath,Bug Measurer (Activity),Cart Balancer (Assessment),...,Tree Top City - Level 2,Tree Top City - Level 3,Watering Hole (Activity),Welcome to Lost Lagoon!,game_session,title,num_correct,num_incorrect,accuracy,accuracy_group
0,0006a69f,True,True,True,False,True,True,True,True,False,...,True,True,True,True,6bdf9623adc94d89,Mushroom Sorter (Assessment),1,0,1.0,3
1,0006a69f,True,True,True,False,True,True,True,True,False,...,True,True,True,True,77b8ee947eb84b4e,Bird Measurer (Assessment),0,11,0.0,0
2,0006a69f,True,True,True,False,True,True,True,True,False,...,True,True,True,True,901acc108f55a5a1,Mushroom Sorter (Assessment),1,0,1.0,3
3,0006a69f,True,True,True,False,True,True,True,True,False,...,True,True,True,True,9501794defd84e4d,Mushroom Sorter (Assessment),1,1,0.5,2
4,0006a69f,True,True,True,False,True,True,True,True,False,...,True,True,True,True,a9ef3ecb3d1acc6a,Bird Measurer (Assessment),1,0,1.0,3


In [217]:
import numpy as np
# Labels are the values we want to predict
labels = np.array(features['accuracy_group'])
# Remove the labels from the features
# axis 1 refers to the columns
features= features.drop(['accuracy_group','accuracy','num_correct','num_incorrect','installation_id','game_session','title'], axis = 1)
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

In [218]:
features

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False,  True]])

In [219]:
labels

array([3, 0, 3, ..., 1, 0, 1])

In [220]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [221]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)


Training Features Shape: (13267, 44)
Training Labels Shape: (13267,)
Testing Features Shape: (4423, 44)
Testing Labels Shape: (4423,)


In [222]:
# # The baseline predictions are the historical averages
# baseline_preds = test_features[:, feature_list.index('average')]
# # Baseline errors, and display average baseline error
# baseline_errors = abs(baseline_preds - test_labels)
# print('Average baseline error: ', round(np.mean(baseline_errors), 2))


In [223]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [224]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# # Calculate the absolute errors
# errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
# print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')


In [225]:
predictions

array([1.65355146, 0.14569843, 1.17205544, ..., 0.45411115, 2.07113322,
       2.31526172])

In [226]:
predictions_formatted=predictions.round()
labels_formatted=test_labels.round()

In [227]:
labels_formatted_pd=pd.DataFrame(data=labels_formatted,columns=[['labels']])

In [228]:
predictions_formatted_pd=pd.DataFrame(predictions_formatted,columns=[['predictions']])

In [229]:
predictions_formatted_pd

,predictions
0,2.0
1,0.0
2,1.0
3,0.0
4,3.0
...,...
4418,2.0
4419,2.0
4420,0.0
4421,2.0


In [230]:
combined=pd.concat([predictions_formatted_pd,labels_formatted_pd],axis=1)

In [231]:
combined.columns=['predictions', 'labels']

In [232]:
combined

,predictions,labels
0,2.0,3
1,0.0,3
2,1.0,2
3,0.0,0
4,3.0,3
...,...,...
4418,2.0,3
4419,2.0,3
4420,0.0,3
4421,2.0,3


In [233]:
labels.round()

array([3, 0, 3, ..., 1, 0, 1])

In [234]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4423 entries, 0 to 4422
Data columns (total 2 columns):
predictions    4423 non-null float64
labels         4423 non-null int64
dtypes: float64(1), int64(1)
memory usage: 69.2 KB


In [164]:
combined.columns

Index(['predictions', 'labels'], dtype='object')

In [235]:
comparison=combined['predictions']==combined['labels']

In [236]:
comparison.sum()/len(comparison)

0.24553470495139046